# Notebook 2.5.4: The Freshman Plague

---

<br>

*Modeling and Simulation in Python*

Copyright 2021 Allen Downey, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/))

Revised, Mike Augspurger (2021-present)

<br>

---

The code cell below contains the `run_simulation` and `change_func` from the previous notebook.

In [ ]:
import pandas as pd
import numpy as np

def change_func(t, state, system):
    s, i, r = state.s, state.i, state.r

    infected = system['C'] * i * s
    recovered = system['R'] * i

    s -= infected
    i += infected - recovered
    r += recovered

    return pd.Series(dict(s=s, i=i, r=r),name="State")

def run_simulation(system, change_func):
    state = pd.Series(dict(s=system['iS'], i=system['iI'], r=system['iR']),
                      name="State Variables")
    frame = pd.DataFrame([],columns=state.index)
    frame.loc[0] = state

    for t in range(0, system['t_end']):
        frame.loc[t+1] = change_func(t, frame.loc[t], system)
    return frame

def calc_total_infected(results, system):
    s_0 = results.s[0]
    s_end = results.s[system['t_end']]
    return s_0 - s_end

def add_immunization(system, fraction):
    system['iS'] -= fraction
    system['iR'] += fraction

In [ ]:

# Create a system
iS = 89; iI = 1; iR = 0
time_c = 3; time_r = 4; t_end = 14*7
N = iS + iI + iR

system = dict(iS=iS/N, iI=iI/N, iR=iR/N, N=N, C = 1/time_c,
              R = 1/time_r, t_end=t_end)
system

## Hand washing

Suppose you are the Dean of Student Life, and you have a budget of just \$1200 to combat the Freshman Plague. You have two options for spending this money:

1.  You can pay for vaccinations, at a rate of \$100 per dose.

2.  You can spend money on a campaign to remind students to wash hands
    frequently.

We have already seen how we can model the effect of vaccination. Now
let's think about the hand-washing campaign. We'll have to answer two
questions:

1.  How should we incorporate the effect of hand washing in the model?

2.  How should we quantify the effect of the money we spend on a
    hand-washing campaign?

For the sake of simplicity, let's assume that we have data from a
similar campaign at another school showing that a well-funded campaign
can change student behavior enough to reduce the infection rate by 20%.

<br>

In terms of the model, hand washing has the effect of reducing `beta`: because students are less likely to infect others, then we assume there are fewer contacts between students that could result in infection.
That's not the only way we could incorporate the effect, but it seems
reasonable and it's easy to implement.

<br>

Now we have to model the relationship between the money we spend and the
effectiveness of the campaign. Again, let's suppose we have data from
another school that suggests:

-   If we spend \$500 on posters, materials, and staff time, we can
    change student behavior in a way that decreases the effective value of `beta` by 10%.

-   If we spend \$1000, the total decrease in `beta` is almost 20%.

-   Above \$1000, additional spending has little additional benefit.

## Choosing a function to model benefits of hand-washing

We have made an assumption that spending more money on a hand-washing campaign will decrease `beta`.   But we need to define that relationship: how much does each new dollar spent change the parameter?

<br>

As with other modeling decisions, this one requires that we make some assumptions (see the assumptions in the previous cell, for instance).  Our function just quantizes these assumptions.   We could assume a linear function: each dollar increases `beta` by a given amount, until we reach 1000 dollars.  This function would look like this, where `D` is the number of dollars spent and $\Delta \beta$ is the change in `beta`:

<br>

$$If D < 1000, \Delta \beta  (D) = (0.20D)/1000$$

<br>

$$ If D  >= 1000, \Delta \beta  (D) = 0.20$$

<br>

But let's use a model that fits our intuition better.   Let's assume that spending a couple dollars would have little benefit: that is, benefits wouldn't really occur until you spent at least a couple hundred dollars.  And further, let's assume that the benefit would gradually decrease as spending approached \$1000.

<br>

One possible function that fits this description is a [generalized logistic function](https://en.wikipedia.org/wiki/Generalised_logistic_function) (GLF), which is a convenient function for modeling curves that have a generally sigmoid shape.  The parameters of the GLF correspond to various features of the curve in a way that makes it easy to find a function that has the shape you want, based on data or background information about the scenario.  Here's a python function that creates a logistic curve:

In [ ]:
from numpy import exp

def logistic(x, A=0, B=1, C=1, M=0, K=1, Q=1, nu=1):
    """Computes the generalize logistic function.

    A: controls the lower bound
    B: controls the steepness of the transition
    C: not all that useful, AFAIK
    M: controls the location of the transition
    K: controls the upper bound
    Q: shift the transition left or right
    nu: affects the symmetry of the transition

    returns: float or array
    """
    exponent = -B * (x - M)
    denom = C + Q * exp(exponent)
    return A + (K-A) / denom ** (1/nu)

Now we need a function that creates the particular logistic function that we want.  We can choose the parameters to fit our assumptions about the effects of spending money on hand-washing:

`M` is chosen so the transition happens around \$500.

`K` is the maximum reduction in `beta`, 20\%.

`B` is chosen by trial and error to yield a curve that seems feasible.

`compute_factor` computes the reduction in `beta` for a given level of campaign spending.  It will return a `beta` reduction from 0\% to 20\% based upon the level of spending:

In [ ]:
def compute_factor(spending):
    """Reduction factor as a function of spending.

    spending: dollars from 0 to 1200

    returns: fractional reduction in beta
    """
    return logistic(spending, M=500, K=0.2, B=0.01)

Let's see what our curve looks like.  We create a `linspace` of possible spending amounts from 0 to 1200 dollars, convert our array to a percent, and then plot it:

In [ ]:
spending = np.linspace(0, 1200, 21)

percent_reduction = compute_factor(spending) * 100

spend_vs_reduce = pd.Series(data=percent_reduction,index=spending)

spend_vs_reduce.plot(xlabel='Hand-washing campaign spending (USD)',
         ylabel='Percent reduction in infection rate',
         title='Effect of hand washing on infection rate');

Now we need to use `compute_factor` to change the `beta` for a particular simulation.

`add_hand_washing` takes a
systems object and a budget, and modifies `system['beta']` to model the
effect of hand washing:

In [ ]:
def add_hand_washing(system, spending):
    factor = compute_factor(spending)
    system['C'] *= (1 - factor)

Now we can sweep a range of values for `spending` and use the simulation
to compute the effect:

In [ ]:
def sweep_hand_washing(system, spending_array):
    sweep = pd.Series([],dtype=object)

    # Unpack variables from original system and return to argument state
    N = system['N']
    iS, iI, iR = system['iS'], system['iI'],system['iR']
    C, R, t_end= system['C'], system['R'], system['t_end']

    for spending in spending_array:
        system_local = dict(iS=iS, iI=iI, iR=iR, N=N, C=C, R=R, t_end=t_end)
        add_hand_washing(system_local, spending)
        results = run_simulation(system_local, change_func)
        sweep[spending] = calc_total_infected(results, system_local)

    return sweep

In [ ]:
from numpy import linspace

# Create the spending range through which we'll sweep
spending_array = linspace(0, 1200, 20)
# Calculate the percentage of total infected students for each spending value
infected_sweep2 = sweep_hand_washing(system,spending_array)

The following figure shows the result.

In [ ]:
infected_sweep2.plot(xlabel='Hand-washing campaign spending (USD)',
         ylabel='Total fraction infected',
         title='Effect of hand washing on total infections');

Below \$200, the campaign has little effect.  At \$800 it has a substantial effect, reducing total infections from more than 45% to about 20%.  Above \$800, the additional benefit is small.  Not surprisingly, the results have a sigmoid shape: this is a direct result of the function we used to model the effects!

## Optimization

Let's put it all together. With a fixed budget of \$1200, we have to
decide how many doses of vaccine to buy and how much to spend on the
hand-washing campaign.  Here are the parameters:

In [ ]:
num_students = 90
budget = 1200
price_per_dose = 100
max_doses = int(budget / price_per_dose)
max_doses

The fraction `budget/price_per_dose` might not be an integer. `int` is a
built-in function that converts numbers to integers, rounding down.  We'll sweep the range of possible doses:

In [ ]:
dose_array = np.arange(0,max_doses+1,1)
dose_array

Remember that `arange()` does not include the stop point (which here is 12): this is why we add 1 to `max_doses`.  Then we run the simulation for each element of `dose_array`:

In [ ]:
def sweep_doses(system,dose_array):
    sweep = pd.Series([],dtype=object)

    # Unpack variables from original system and return to argument state
    N = system['N']
    iS, iI, iR = system['iS'], system['iI'],system['iR']
    C, R, t_end= system['C'], system['R'], system['t_end']

    for doses in dose_array:
        fraction = doses / num_students
        spending = budget - doses * price_per_dose

        # Adjust system according to preventive actions
        system_local = dict(iS=iS, iI=iI, iR=iR, N=N, C=C, R=R, t_end=t_end)
        add_immunization(system_local, fraction)
        add_hand_washing(system_local, spending)

        # Find results for each budget option
        results = run_simulation(system_local, change_func)
        sweep[doses] = calc_total_infected(results, system_local)

    return sweep

For each number of doses, we compute the fraction of students we can
immunize, `fraction` and the remaining budget we can spend on the
campaign, `spending`. Then we run the simulation with those quantities
and store the number of infections.

The following figure shows the result.

In [ ]:
infected_sweep3 = sweep_doses(system,dose_array)

In [ ]:
infected_sweep3.plot(xlabel='Doses of vaccine',
         ylabel='Total fraction infected',
         title='Total infections vs. doses');

If we buy no doses of vaccine and spend the entire budget on the campaign, the fraction infected is around 19%. At 4 doses, we have \$800 left for the campaign, and this is the optimal point that minimizes the number of students who get sick.

<br>

As we increase the number of doses, we have to cut campaign spending,
which turns out to make things worse. But interestingly, when we get
above 10 doses, the effect of herd immunity starts to kick in, and the
number of sick students goes down again.

### Exercise 1

Suppose the price of the vaccine drops to $50 per dose.  How does that affect the optimal allocation of the spending?

In [ ]:
# Run and plot the results of sweep_doses here
num_students = 90
budget = 1200
price_per_dose = 50
max_doses = int(budget / price_per_dose)
max_doses

dose_array = np.arange(0,max_doses+1,1)

infected_sweep3 = sweep_doses(system,dose_array)

infected_sweep3.plot(xlabel='Doses of vaccine',
         ylabel='Total fraction infected',
         title='Total infections vs. doses')

### Exercise 2

✅ Explain the plot produced in Exercise 1.  What is the best use of the budget with these new conditions?  Why have these conditions (cheaper vaccine) caused these results?